### util function

In [1]:
# util function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import numpy as np
def standard_input(X):
    # 标准化输入
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return pd.DataFrame(X_scaled, columns=X.columns)



def load_data_SGER(mode='raw',random_state=42, aug_pct=100):
    """
    加载数据并进行预处理，返回以下变量：
    X, y, X_train, X_valid, X_test, y_train, y_valid, y_test, train_mask, valid_mask, test_mask

    参数：
        mode (str): 数据读取模式，可选 'raw' 或 'aug'
            - 'raw': 仅使用 raw 数据（SGER1000.csv），逻辑与原 load_data_SGER_RAW 一致
            - 'aug': raw 数据按原逻辑划分，且读取 LLM 数据（SGERLLM.csv）并将其按 90%/10%分别追加到
                     训练集和验证集中；测试集仍使用 raw 数据划分出的部分
        aug_pct (int 或 float): 使用的增强数据比例，取值范围 0~100，默认 100 表示使用全部 LLM 数据。
                                  当 mode 为 'aug' 时有效。
    """
    # 定义文件路径
    path_raw = '/home/gehongfei/project/TabGNN/dataset/SGER1000.csv'
    path_llm = '/home/gehongfei/project/TabGNN/dataset/SGERLLM.csv'
    
    # ---------------------------
    # 1. 处理 raw 数据
    # ---------------------------
    try:
        df_raw = pd.read_csv(path_raw, sep='\s+')
    except Exception as e:
        print(f"Error reading raw file {path_raw}: {e}")
        return None
    
    if 'kredit' not in df_raw.columns:
        print("Error: 'kredit' 列未在 raw 数据中找到。")
        return None

    # 分离特征和目标变量
    y_raw = df_raw['kredit']
    X_raw = df_raw.drop(columns=['kredit'])
    
    # 划分 raw 数据：70% 为训练集，30% 为临时集合
    X_train_raw, X_temp, y_train_raw, y_temp = train_test_split(
        X_raw, y_raw, test_size=0.3, random_state=random_state, stratify=y_raw
    )
    # 将临时集合再划分为验证集和测试集，其中 1/3（约 10%）为验证集，2/3（约 20%）为测试集
    X_valid_raw, X_test_raw, y_valid_raw, y_test_raw = train_test_split(
        X_temp, y_temp, test_size=2/3, random_state=random_state, stratify=y_temp
    )
    
    # ---------------------------
    # 2. 根据 mode 处理数据
    # ---------------------------
    if mode == 'raw':
        # 直接使用 raw 数据划分结果
        X_train_final = X_train_raw.reset_index(drop=True)
        y_train_final = y_train_raw.reset_index(drop=True)
        X_valid_final = X_valid_raw.reset_index(drop=True)
        y_valid_final = y_valid_raw.reset_index(drop=True)
        X_test_final  = X_test_raw.reset_index(drop=True)
        y_test_final  = y_test_raw.reset_index(drop=True)
        
    elif mode == 'aug':
        # 读取 LLM 数据，并将数据追加到 raw 的训练集和验证集中
        try:
            df_llm = pd.read_csv(path_llm, sep='\s+')
        except Exception as e:
            print(f"Error reading LLM file {path_llm}: {e}")
            return None
        
        if 'kredit' not in df_llm.columns:
            print("Error: 'kredit' 列未在 LLM 数据中找到。")
            return None
        
        # 根据 aug_pct 参数调整使用的增强数据比例
        if aug_pct < 100:
            df_llm = df_llm.sample(frac=aug_pct/100, random_state=42).reset_index(drop=True)
        
        # 分离 LLM 数据中的特征和目标变量
        y_llm = df_llm['kredit']
        X_llm = df_llm.drop(columns=['kredit'])
        
        # 划分 LLM 数据：90% 为训练，10% 为验证
        X_llm_train, X_llm_valid, y_llm_train, y_llm_valid = train_test_split(
            X_llm, y_llm, test_size=0.1, random_state=42, stratify=y_llm
        )
        
        # 重置索引，防止后续合并时出现冲突
        X_train_raw = X_train_raw.reset_index(drop=True)
        y_train_raw = y_train_raw.reset_index(drop=True)
        X_valid_raw = X_valid_raw.reset_index(drop=True)
        y_valid_raw = y_valid_raw.reset_index(drop=True)
        X_llm_train = X_llm_train.reset_index(drop=True)
        y_llm_train = y_llm_train.reset_index(drop=True)
        X_llm_valid = X_llm_valid.reset_index(drop=True)
        y_llm_valid = y_llm_valid.reset_index(drop=True)
        X_test_raw  = X_test_raw.reset_index(drop=True)
        y_test_raw  = y_test_raw.reset_index(drop=True)
        
        # 追加 LLM 数据：训练集增加 LLM 训练数据，验证集增加 LLM 验证数据
        X_train_final = pd.concat([X_train_raw, X_llm_train], axis=0).reset_index(drop=True)
        y_train_final = pd.concat([y_train_raw, y_llm_train], axis=0).reset_index(drop=True)
        X_valid_final = pd.concat([X_valid_raw, X_llm_valid], axis=0).reset_index(drop=True)
        y_valid_final = pd.concat([y_valid_raw, y_llm_valid], axis=0).reset_index(drop=True)
        # 测试集仍使用 raw 划分出的数据
        X_test_final  = X_test_raw
        y_test_final  = y_test_raw
    else:
        print("Error: mode 必须为 'raw' 或 'aug'")
        return None

    # ---------------------------
    # 3. 合并所有数据，并构造掩码
    # ---------------------------
    # 合并训练、验证、测试数据为完整数据集
    X = pd.concat([X_train_final, X_valid_final, X_test_final], axis=0).reset_index(drop=True)
    y = pd.concat([y_train_final, y_valid_final, y_test_final], axis=0).reset_index(drop=True)
    
    # 构造掩码，保证返回的变量名与要求一致
    num_total = len(X)
    train_mask = torch.zeros(num_total, dtype=torch.bool)
    valid_mask = torch.zeros(num_total, dtype=torch.bool)
    test_mask  = torch.zeros(num_total, dtype=torch.bool)
    
    # 训练集占合并数据的前部分
    len_train = len(X_train_final)
    train_mask[:len_train] = True
    # 验证集紧随训练集之后
    len_valid = len(X_valid_final)
    valid_mask[len_train:len_train+len_valid] = True
    # 测试集为剩余部分
    test_mask[len_train+len_valid:] = True

    # ---------------------------
    # 4. 标准化数据
    # ---------------------------
    X = standard_input(X)
    X_train = standard_input(X_train_final)
    X_valid = standard_input(X_valid_final)
    X_test  = standard_input(X_test_final)
    
    return X, y, X_train, X_valid, X_test, \
           y_train_final, y_valid_final, y_test_final, \
           train_mask, valid_mask, test_mask

# 训练Random Forest并计算相似性（完整相似性矩阵）
def compute_adjacency_matrix(X_train, X_valid, X_test, y_train, y_valid, n_estimators=200, max_depth=None, random_state=42):
    # 合并训练、验证和测试数据
    X_combined = pd.concat([X_train, X_valid, X_test], axis=0)
    num_samples = X_combined.shape[0]

    # 训练Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    rf.fit(pd.concat([X_train, X_valid], axis=0), pd.concat([y_train, y_valid], axis=0))

    # 获取每棵树的叶子索引
    leaf_indices = rf.apply(X_combined)

    # 计算完整的相似性矩阵
    adjacency_matrix = np.zeros((num_samples, num_samples))
    for tree_idx in range(leaf_indices.shape[1]):  # 遍历每棵树
        leaf_to_samples = {}
        for sample_idx, leaf_id in enumerate(leaf_indices[:, tree_idx]):
            if leaf_id not in leaf_to_samples:
                leaf_to_samples[leaf_id] = []
            leaf_to_samples[leaf_id].append(sample_idx)
        # 更新相似性矩阵
        for sample_list in leaf_to_samples.values():
            for i in sample_list:
                for j in sample_list:
                    if i != j:
                        adjacency_matrix[i, j] += 1

    # 归一化相似性
    adjacency_matrix /= adjacency_matrix.max()

    # 返回完整的邻接矩阵（不进行二值化）
    return csr_matrix(adjacency_matrix)


# 从稀疏邻接矩阵提取边索引（并在这里应用 threshold 进行二值化）
def adjacency_to_edge_index(adj_matrix, threshold=0.15):
    adj_matrix = adj_matrix.toarray()  # 转换为 NumPy 数组
    adj_matrix = (adj_matrix > threshold).astype(int)  # 在这里应用阈值二值化
    coo_matrix = csr_matrix(adj_matrix).tocoo()  # 转换为COO格式
    edge_index = torch.tensor(np.vstack((coo_matrix.row, coo_matrix.col)), dtype=torch.long)
    return edge_index


# 测试数据加载函数
X, y, X_train, X_valid, X_test, y_train, y_valid, y_test, train_mask, valid_mask, test_mask = load_data_SGER()
print("数据加载完成！")

# 计算完整的邻接矩阵（不进行二值化）
print("计算完整的邻接矩阵中...")
adjacency_matrix = compute_adjacency_matrix(X_train, X_valid, X_test, y_train, y_valid)
print("完整邻接矩阵计算完成！")

# 将邻接矩阵转换为 PyTorch 格式，并在这里应用 threshold
threshold = 0.15  # 设置阈值
edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
print("邻接矩阵转换完成！")
print("Edge index 维度:", edge_index.shape)


数据加载完成！
计算完整的邻接矩阵中...
完整邻接矩阵计算完成！
邻接矩阵转换完成！
Edge index 维度: torch.Size([2, 42652])


### add unsupervised contrastive loss NodeDropout

In [4]:
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.utils import k_hop_subgraph

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#########################################
# 1. 定义模型
#########################################
# GraphSAGE 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率
        """
        super(GraphSAGE, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层：从 in_channels 到 hidden_channels
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        if in_channels != hidden_channels:
            self.residuals.append(torch.nn.Linear(in_channels, hidden_channels))
        else:
            self.residuals.append(torch.nn.Identity())
        current_hidden = hidden_channels
        # 后续每一层的隐藏单元数为上一层的 3/4（向下取整，最小为 1）
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_hidden * 3 / 4))
            self.convs.append(SAGEConv(current_hidden, next_hidden))
            if current_hidden != next_hidden:
                self.residuals.append(torch.nn.Linear(current_hidden, next_hidden))
            else:
                self.residuals.append(torch.nn.Identity())
            current_hidden = next_hidden
        # 全连接层：将最后一层的隐藏向量映射到输出类别
        self.fc = torch.nn.Linear(current_hidden, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过图卷积层、残差连接、ReLU 和 dropout（适用于全图或子图）"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

# GAT 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5, heads=1, concat=True):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率（同时用于 GATConv 内部 dropout）
          heads: 注意力头的数量
          concat: 是否拼接多头输出（True）或取平均（False）
        """
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout_rate, concat=concat))
        out_dim = hidden_channels * heads if concat else hidden_channels
        if in_channels != out_dim:
            self.residuals.append(torch.nn.Linear(in_channels, out_dim))
        else:
            self.residuals.append(torch.nn.Identity())
        current_dim = out_dim
        # 后续层
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_dim * 3 / 4))
            self.convs.append(GATConv(current_dim, next_hidden, heads=heads, dropout=dropout_rate, concat=concat))
            new_out_dim = next_hidden * heads if concat else next_hidden
            if current_dim != new_out_dim:
                self.residuals.append(torch.nn.Linear(current_dim, new_out_dim))
            else:
                self.residuals.append(torch.nn.Identity())
            current_dim = new_out_dim
        self.fc = torch.nn.Linear(current_dim, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过 GAT 层、残差连接、ReLU 和 dropout"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

#########################################
# 2. 定义损失函数
#########################################
# Focal Loss（用于处理类别不平衡）
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  
        self.reduction = reduction
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            if isinstance(self.alpha, (list, np.ndarray)):
                alpha = inputs.new_tensor(self.alpha)
            else:
                alpha = self.alpha
            at = alpha.gather(0, targets.data)
            ce_loss = at * ce_loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean() if self.reduction == "mean" else focal_loss.sum()

# 修改后的普通对比学习损失（不使用 mask）
class SupConLoss(torch.nn.Module):
    def __init__(self, temperature=0.07):
        """
        Args:
            temperature: 温度参数
        """
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features):
        """
        Args:
            features: [batch_size, n_views, feature_dim]
                      要求每个样本至少有两个视图，视图之间互为正样本，其余样本均为负样本。
        Returns:
            对比损失（InfoNCE Loss）
        """
        device = features.device
        if len(features.shape) < 3:
            raise ValueError('`features` 需要形状为 [batch_size, n_views, feature_dim]')
        batch_size, n_views, feature_dim = features.shape

        # 将多个视图拼接为 [batch_size*n_views, feature_dim]
        features = features.view(batch_size * n_views, feature_dim)
        # 对每个特征进行 L2 归一化
        features = torch.nn.functional.normalize(features, p=2, dim=1)

        # 计算相似度矩阵，形状 [batch_size*n_views, batch_size*n_views]
        similarity_matrix = torch.matmul(features, features.T) / self.temperature

        # 构造正样本掩码：同一原始样本（即同一 batch 中的不同视图）的两两之间为正样本
        labels = torch.arange(batch_size, device=device).repeat_interleave(n_views)
        mask = torch.eq(labels.unsqueeze(1), labels.unsqueeze(0)).float()
        # 去除自身对比（对角线置 0）
        self_mask = torch.eye(mask.shape[0], device=device)
        mask = mask - self_mask

        # 计算 exp(similarity)
        exp_sim = torch.exp(similarity_matrix) * (1 - self_mask)
        # 对每个 anchor，分母为除自身外所有样本的 exp(sim)
        denom = exp_sim.sum(dim=1, keepdim=True) + 1e-8

        # 计算仅正样本对的对数概率
        log_prob = similarity_matrix - torch.log(denom)
        numerator = (mask * log_prob).sum(dim=1)
        # 正样本个数（防止除 0）
        pos_count = mask.sum(dim=1) + 1e-8
        loss = - (numerator / pos_count)
        loss = loss.mean()
        return loss

#########################################
# 3. 数据增强方法
#########################################
# 原始的特征扰动（用于 "feature" 增强方式）
def perturb_features(features, noise_level=0.1):
    """对特征进行扰动，生成增强视图"""
    noise = torch.randn_like(features) * noise_level
    return features + noise

# 节点丢弃
def augment_node_drop(features, edge_index, drop_prob=0.1):
    """
    节点丢弃：以一定概率丢弃节点（将被丢弃节点的特征置零，
    同时删除其相关边，但保持节点的序号不变）。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_nodes = features.shape[0]
    keep_mask = (torch.rand(num_nodes, device=features.device) > drop_prob)
    features_aug = features * keep_mask.unsqueeze(1).float()
    src, dst = edge_index
    valid_edge_mask = keep_mask[src] & keep_mask[dst]
    edge_index_aug = edge_index[:, valid_edge_mask]
    return features_aug, edge_index_aug

# 边丢弃
def augment_edge_drop(features, edge_index, drop_prob=0.1):
    """
    边丢弃：以一定概率删除边，但保留所有节点和原始特征。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_aug = edge_index[:, mask]
    return features, edge_index_aug

# 边扰动
def augment_edge_perturb(features, edge_index, drop_prob=0.1):
    """
    边扰动：先以一定概率删除部分边，再随机添加一些新的边，
    添加的新边数量与被删除边的数量相当。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    num_nodes = features.shape[0]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_dropped = edge_index[:, mask]
    num_dropped = num_edges - mask.sum().item()
    if num_dropped > 0:
        new_edges = torch.randint(0, num_nodes, (2, num_dropped), device=features.device)
        edge_index_aug = torch.cat([edge_index_dropped, new_edges], dim=1)
    else:
        edge_index_aug = edge_index_dropped
    return features, edge_index_aug

def augment_data(data, aug_method="feature", aug_ratio=0.1):
    """
    根据指定的增强方式对图数据进行增强，返回增强后的节点特征和 edge_index。
    参数:
      aug_method: "feature"（特征扰动）, "node_drop", "edge_drop", "edge_perturb"
      aug_ratio: 控制增强强度（例如噪声水平或丢弃比例）
    """
    if aug_method == "feature":
        x_aug = perturb_features(data.x, noise_level=aug_ratio)
        edge_index_aug = data.edge_index  # 图结构不变
        return x_aug, edge_index_aug
    elif aug_method == "node_drop":
        return augment_node_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_drop":
        return augment_edge_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_perturb":
        return augment_edge_perturb(data.x, data.edge_index, drop_prob=aug_ratio)
    else:
        raise ValueError(f"Unknown augmentation method: {aug_method}")

#########################################
# 3.1 辅助函数：提取 mini-batch 子图
#########################################
def get_mini_batch_data(data, batch_node_idx, num_hops):
    """
    对单图中一小批节点（batch_node_idx）提取 k-hop 子图。
    使用 torch_geometric.utils.k_hop_subgraph 进行子图提取，并 relabel 节点。
    返回：
      sub_data: 包含子图的 Data 对象（x, edge_index, y 以及 mask 可选）
      mapping: 一个长整型张量，指示子图中哪一部分对应原始 batch_node_idx（目标节点在子图中的索引）
    """
    # k_hop_subgraph 返回：(subset, sub_edge_index, mapping, edge_mask)
    subset, sub_edge_index, mapping, _ = k_hop_subgraph(
        node_idx=batch_node_idx, num_hops=num_hops, edge_index=data.edge_index, relabel_nodes=True)
    sub_data = Data(x=data.x[subset], edge_index=sub_edge_index, y=data.y[subset])
    # 如果原始 data 定义了 train/val/test mask，则在子图中也保留（注意：mask 按 subset 索引提取）
    if hasattr(data, 'train_mask'):
        sub_data.train_mask = data.train_mask[subset]
    if hasattr(data, 'val_mask'):
        sub_data.val_mask = data.val_mask[subset]
    if hasattr(data, 'test_mask'):
        sub_data.test_mask = data.test_mask[subset]
    return sub_data, mapping

#########################################
# 4. 训练函数（预训练 + 微调）——mini-batch 版（基于 k_hop_subgraph）
#########################################
def pretrain_model(data, model, optimizer, criterion_contrast, num_epochs=200, aug_method="feature", aug_ratio=0.1, batch_size=64):
    """
    预训练阶段：仅使用对比损失训练模型（不计算 Focal Loss）。
    采用 mini-batch 方式，先对训练集中的一批目标节点提取 k-hop 子图，
    再对该子图进行数据增强生成两视图，最后计算对比损失（仅对 batch 中的目标节点计算）。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)  # 以模型层数作为子图的 hop 数

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        for batch in loader:
            # 如果 batch 已经是 tensor，则使用 clone().detach().to(device)
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            x_aug1, edge_index1 = augment_data(sub_data, aug_method, aug_ratio)
            x_aug2, edge_index2 = augment_data(sub_data, aug_method, aug_ratio)
            embedding_aug1 = model.encode(x_aug1, edge_index1)
            embedding_aug2 = model.encode(x_aug2, edge_index2)
            target_emb1 = embedding_aug1[mapping]
            target_emb2 = embedding_aug2[mapping]
            features_aug = torch.stack([target_emb1, target_emb2], dim=1)
            loss = criterion_contrast(features_aug)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * len(batch)
            count += len(batch)
        avg_loss = total_loss / count
        print(f"Pretrain Epoch: {epoch}, Contrast Loss: {avg_loss:.4f}")

        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state

def fine_tune_model(data, model, optimizer, criterion_focal, num_epochs=50, batch_size=64):
    """
    微调阶段：仅使用 Focal Loss 进行训练（不计算对比损失）。
    同样采用 mini-batch（基于 k_hop 子图）方式训练，损失仅在目标节点上计算。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        for batch in loader:
            # 使用 clone().detach().to(device) 避免不必要的拷贝警告
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            out = model(sub_data)
            loss = criterion_focal(out[mapping], sub_data.y[mapping])
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * len(batch)
            count += len(batch)
        avg_loss = total_loss / count
        print(f"Fine-tune Epoch: {epoch}, Focal Loss: {avg_loss:.4f}")

        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state


def two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_focal, criterion_contrast,
                          pretrain_epochs, finetune_epochs, aug_method="feature", aug_ratio=0.1,
                          batch_size=64):
    """
    两阶段训练：
      第一阶段：预训练（仅用对比损失，mini-batch 方式）；
      第二阶段：微调（仅用分类损失，mini-batch 方式）。
    """
    print("========== 开始预训练阶段 ==========")
    best_pretrain_state = pretrain_model(data, model, optimizer, criterion_contrast,
                                         num_epochs=pretrain_epochs,
                                         aug_method=aug_method, aug_ratio=aug_ratio,
                                         batch_size=batch_size)
    model.load_state_dict(best_pretrain_state)

    print("========== 开始微调阶段 ==========")
    best_finetune_state = fine_tune_model(data, model, optimizer_ft,
                                          criterion_focal, num_epochs=finetune_epochs,
                                          batch_size=batch_size)
    return best_finetune_state

#########################################
# 5. 封装随机采样超参数组合的函数
#########################################
import random

def get_continuous_candidates(start, stop, step, decimals):
    """
    生成从 start 到 stop（含）之间，以 step 为步长的候选列表，并保留指定小数位数。
    """
    num_steps = int((stop - start) / step) + 1
    return [round(start + i * step, decimals) for i in range(num_steps)]

def get_random_hyperparameter_combinations(n_iter):
    """
    随机生成 n_iter 个超参数组合，每个组合包含：
      (threshold, random_state, num_layers, hidden_channels, finetune_lr, pretrain_lr,
       gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
       model_type, dropout_rate)
    """
    # 离散变量候选列表
    discrete_candidates = {
        'random_state': list(range(0, 40, 10)),
        'num_layers': [2, 3, 4],
        'hidden_channels': list(range(20, 300, 20)),
        'finetune_lr': [0.005, 0.01],
        'pretrain_lr': [0.0001, 0.001, 0.000001],
        'aug_method': ["feature", "node_drop", "edge_drop", "edge_perturb"],
        'pretrain_epochs': [10],
        'model_type': ["GraphSAGE", "GAT"],
        'dropout_rate': [0, 0.1, 0.2, 0.3]
    }

    # 连续变量候选区间及步长（注意：部分变量保留 2 位小数，其余保留 1 位小数）
    continuous_candidates = {
        'threshold': get_continuous_candidates(0.1, 0.3, 0.01, 2),
        'gamma': get_continuous_candidates(2, 4, 0.1, 1),
        'alpha_value': get_continuous_candidates(0.3, 0.6, 0.1, 1),
        'aug_ratio': get_continuous_candidates(0.05, 0.25, 0.01, 2),
        'temperature': get_continuous_candidates(0.05, 0.3, 0.01, 2)
    }

    combinations = []
    for _ in range(n_iter):
        # 随机采样连续变量
        threshold   = random.choice(continuous_candidates['threshold'])
        gamma       = random.choice(continuous_candidates['gamma'])
        alpha_value = random.choice(continuous_candidates['alpha_value'])
        aug_ratio   = random.choice(continuous_candidates['aug_ratio'])
        temperature = random.choice(continuous_candidates['temperature'])
        
        # 随机采样离散变量
        random_state   = random.choice(discrete_candidates['random_state'])
        num_layers     = random.choice(discrete_candidates['num_layers'])
        hidden_channels= random.choice(discrete_candidates['hidden_channels'])
        finetune_lr    = random.choice(discrete_candidates['finetune_lr'])
        pretrain_lr    = random.choice(discrete_candidates['pretrain_lr'])
        aug_method     = random.choice(discrete_candidates['aug_method'])
        pretrain_epochs= random.choice(discrete_candidates['pretrain_epochs'])
        model_type     = random.choice(discrete_candidates['model_type'])
        dropout_rate   = random.choice(discrete_candidates['dropout_rate'])
        
        # 构造超参数组合（顺序与注释中保持一致）
        combination = (
            threshold,      # 阈值
            random_state,   # 随机种子
            num_layers,     # 图卷积层数
            hidden_channels,# 第一层隐藏单元数
            finetune_lr,    # 微调学习率
            pretrain_lr,    # 预训练学习率
            gamma,          # gamma 参数
            alpha_value,    # alpha 参数
            aug_method,     # 增强方式
            aug_ratio,      # 增强比例
            pretrain_epochs,# 预训练轮数
            temperature,    # 对比学习温度
            model_type,     # 模型类型
            dropout_rate    # dropout 概率
        )
        combinations.append(combination)
        
    return combinations

# 示例：生成 5 个超参数组合
if __name__ == '__main__':
    combos = get_random_hyperparameter_combinations(5)
    for combo in combos:
        print(combo)


#########################################
# 6. 随机搜索超参数并评估模型
#########################################
def grid_search(X, y, train_mask, valid_mask, test_mask, n_iter):
    best_acc = 0.0
    best_overall_model_state = None
    best_overall_params = None

    print("Start random search with {} combinations...".format(n_iter))
    hyperparam_combos = get_random_hyperparameter_combinations(n_iter)
    for i, (threshold, random_state, num_layers, hidden_channels, finetune_lr,
            pretrain_lr, gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
            model_type, dropout_rate) in enumerate(hyperparam_combos):
        print(f"\nTesting combination {i+1}: threshold={threshold:.4f}, random_state={random_state}, "
              f"layers={num_layers}, hidden_channels={hidden_channels}, finetune_lr={finetune_lr}, "
              f"pretrain_lr={pretrain_lr}, gamma={gamma:.4f}, alpha={alpha_value:.4f}, aug_method={aug_method}, "
              f"aug_ratio={aug_ratio:.4f}, pretrain_epochs={pretrain_epochs}, temperature={temperature:.4f}, "
              f"model_type={model_type}, dropout_rate={dropout_rate:.4f}")



        X_tensor = torch.tensor(X.values, dtype=torch.float)
        y_tensor = torch.tensor(y.values, dtype=torch.long)
        edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
        data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index,
                    train_mask=train_mask, val_mask=valid_mask, test_mask=test_mask).to(device)

        # 根据 model_type 选择模型
        if model_type == "GraphSAGE":
            model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                              out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        elif model_type == "GAT":
            model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                        out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_lr, weight_decay=5e-4)
        optimizer_ft = torch.optim.Adam(model.parameters(), lr=finetune_lr, weight_decay=5e-4)

        alpha_list = [1 - alpha_value, alpha_value]
        alpha_tensor = torch.tensor(alpha_list, dtype=torch.float).to(device)
        criterion_focal = FocalLoss(gamma=gamma, alpha=alpha_tensor, reduction="mean")
        criterion_contrast = SupConLoss(temperature=temperature)

        best_model_epoch = two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_focal,
                                                 criterion_contrast, pretrain_epochs=pretrain_epochs, finetune_epochs=10,
                                                 aug_method=aug_method, aug_ratio=aug_ratio, batch_size=512)
        model.load_state_dict(best_model_epoch)
        model.eval()
        with torch.no_grad():
            test_out = model(data)
            preds = test_out[data.test_mask].argmax(dim=1)
            test_acc = accuracy_score(data.y[data.test_mask].cpu(), preds.cpu())
        print(f"Test Accuracy for current combination: {test_acc:.4f}")
        if test_acc > best_acc:
            best_acc = test_acc
            best_overall_model_state = best_model_epoch
            best_overall_params = (threshold, random_state, num_layers, hidden_channels,
                                   finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
                                   pretrain_epochs, temperature, model_type, dropout_rate)

    return best_overall_params, best_overall_model_state

#########################################
# 7. 主程序：加载数据、随机搜索超参数、加载最佳模型并评估
#########################################
# 注意：此处假定 X, y, edge_index, train_mask, valid_mask, test_mask 已经提前加载好，
# 且 edge_index 为形状 [2, num_edges] 的 torch.tensor 对象。
best_params, best_model_state = grid_search(X, y, train_mask, valid_mask, test_mask, n_iter=400)
print("\nBest Hyperparameters:", best_params)

# 解包最佳超参数
(threshold, random_state, num_layers, hidden_channels,
 finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
 pretrain_epochs, temperature, model_type, dropout_rate) = best_params

edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
X_tensor = torch.tensor(X.values, dtype=torch.float).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.long).to(device)
data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index,
            train_mask=train_mask.to(device), val_mask=valid_mask.to(device), test_mask=test_mask.to(device))

# 根据最终选出的模型类型构造模型
if model_type == "GraphSAGE":
    model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                      out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
elif model_type == "GAT":
    model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
else:
    raise ValueError(f"Unknown model type: {model_type}")

model.load_state_dict(best_model_state)
model.eval()

with torch.no_grad():
    test_out = model(data)
    preds = test_out[data.test_mask].argmax(dim=1)
    true_labels = data.y[data.test_mask]
    report = classification_report(true_labels.cpu(), preds.cpu(), target_names=["Class 0", "Class 1"], digits=4)
    test_precision = precision_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_recall = recall_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_f1 = f1_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_acc = accuracy_score(true_labels.cpu(), preds.cpu())
    
    print("\nBest Model Classification Report on Test Set:")
    print(report)
    print("Best Model Test Set Metrics:")
    print(f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, Accuracy: {test_acc:.4f}")


(0.19, 30, 3, 80, 0.01, 0.001, 3.7, 0.5, 'feature', 0.07, 10, 0.06, 'GAT', 0.1)
(0.23, 20, 3, 240, 0.005, 0.001, 3.4, 0.5, 'node_drop', 0.17, 10, 0.12, 'GAT', 0.2)
(0.19, 10, 4, 100, 0.01, 0.0001, 3.1, 0.5, 'edge_drop', 0.11, 10, 0.22, 'GraphSAGE', 0.1)
(0.26, 30, 2, 180, 0.01, 1e-06, 2.1, 0.5, 'feature', 0.05, 10, 0.1, 'GraphSAGE', 0.2)
(0.21, 10, 3, 80, 0.005, 1e-06, 3.1, 0.3, 'feature', 0.22, 10, 0.26, 'GAT', 0)
Start random search with 400 combinations...

Testing combination 1: threshold=0.2000, random_state=10, layers=3, hidden_channels=260, finetune_lr=0.005, pretrain_lr=0.0001, gamma=3.3000, alpha=0.5000, aug_method=node_drop, aug_ratio=0.2500, pretrain_epochs=10, temperature=0.1300, model_type=GraphSAGE, dropout_rate=0.2000
========== 开始预训练阶段 ==========
Pretrain Epoch: 0, Contrast Loss: 6.2792
Pretrain Epoch: 1, Contrast Loss: 6.2095
Pretrain Epoch: 2, Contrast Loss: 6.1064
Pretrain Epoch: 3, Contrast Loss: 6.0834
Pretrain Epoch: 4, Contrast Loss: 6.1211
Pretrain Epoch: 5, Con

In [ ]:
# 使用最佳模型在测试集上进行评估，并输出分类报告和错误样本的索引集合
with torch.no_grad():
    # 预测测试集
    test_out = model(data)
    # 取出测试集对应的预测结果（注意：data.test_mask 为 bool 型 mask）
    preds = test_out[data.test_mask].argmax(dim=1)
    true_labels = data.y[data.test_mask]

    # 输出测试集上的分类报告
    print("Test Set Classification Report:")
    print(classification_report(true_labels.cpu(), preds.cpu(), target_names=["Class 0", "Class 1"], digits=4))

    # 获取测试集在整个数据集中的原始索引（假设 data.test_mask 是一个布尔向量）
    test_indices = torch.where(data.test_mask)[0].cpu().numpy()
    
    # 将预测和真实标签转换为 numpy 数组
    preds_np = preds.cpu().numpy()
    true_np = true_labels.cpu().numpy()

    # 计算四类样本的索引集合：
    # True Positives (TP): 真标签为 1 且预测为 1 （1 被分为 1）
    tp_idx = test_indices[(true_np == 1) & (preds_np == 1)]
    # False Negatives (FN): 真标签为 1 但预测为 0 （1 被分为 0）
    fn_idx = test_indices[(true_np == 1) & (preds_np == 0)]
    # True Negatives (TN): 真标签为 0 且预测为 0 （0 被分为 0）
    tn_idx = test_indices[(true_np == 0) & (preds_np == 0)]
    # False Positives (FP): 真标签为 0 但预测为 1 （0 被分为 1）
    fp_idx = test_indices[(true_np == 0) & (preds_np == 1)]

    print("\nError Samples Index Sets:")
    print("True Positives (1 被分为 1):", tp_idx)
    print("False Negatives (1 被分为 0):", fn_idx)
    print("True Negatives (0 被分为 0):", tn_idx)
    print("False Positives (0 被分为 1):", fp_idx)


### 比较有无对比损失